In [1]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("feature_aggrigated_flairvader_10APR_NOT_BY_HOUR.gz")

In [15]:
df = df.drop(['Unnamed: 0', 'Index'], axis = 1)

In [18]:
cap_col = ['videoID', 'captionString']
cap_df = df[cap_col]

In [19]:
cap_df

,videoID,captionString
0,y4fb_R5Ogw0,so this setup that you're seeing is what we ha...
1,ALEXVtnNEwA,b welcome to my kitchen i hope all of you are ...
2,E98hC9e__Xs,Welcome to my kitchen. I hope all of you are...
3,nMns8t4OhI0,NaN
4,AG_ZHi3tuyk,- The RTX 3080 is the most powerful GPU we've ...
...,...,...
22913,O4kkTlIMDzU,NaN
22914,mjjhe592OHg,and video pulls a move that is good for the co...
22915,iXn9O-Rzb_M,NaN
22916,MrcNkPOX77U,is go magic me is uh and i'm gett...


In [20]:
print("Number of unique videos: {}".format(len(cap_df.videoID.unique())))

Number of unique videos: 500


In [22]:
# get unique videos ID 
cap_df_dedu = cap_df.drop_duplicates(subset = ['videoID'])

In [23]:
cap_df_dedu = cap_df_dedu.copy()
cap_df_dedu.dropna(inplace = True)

In [24]:
cap_df_dedu.reset_index(drop = True, inplace = True)

In [25]:
cap_df_dedu

,videoID,captionString
0,y4fb_R5Ogw0,so this setup that you're seeing is what we ha...
1,ALEXVtnNEwA,b welcome to my kitchen i hope all of you are ...
2,E98hC9e__Xs,Welcome to my kitchen. I hope all of you are...
3,AG_ZHi3tuyk,- The RTX 3080 is the most powerful GPU we've ...
4,ucutmH2KvSQ,(clapboard sounds) - Did you guys just seethe ...
...,...,...
265,L7_KkIjcVIU,what's going down tech fans today nvidia has d...
266,-KAEYCEz3mw,so what is your boys it's me paul from non app...
267,_fyGXaSSCyE,do yeah boy bruh wow hey hahaha uma li...
268,2cMfPzWErYg,well we found out where all the gpus are going...


In [26]:
# sentence embedding with BERTsentence 
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pyOpenSSL 20.0.1 (/Users/apple/opt/anaconda3/lib/python3.7/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyOpenSSL 20.0.1 (/Users/apple/opt/anaconda3/lib/python3.7/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 20.0.1 (/Users/apple/opt/anaconda3/lib/python3.7/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepR

In [62]:
cap_df_dedu.videoID.shape

(270,)

In [31]:
sentence_embeddings = model.encode(cap_df_dedu.captionString)

In [61]:
sentence_embeddings.shape

(270, 768)

In [64]:
id2vec_df = pd.concat([cap_df_dedu.videoID, pd.DataFrame(sentence_embeddings)], axis = 1)

In [69]:
df_vec = df.merge(id2vec_df, on = "videoID")

In [70]:
X_feautures_pre_merge_1.groupby(
    pd.Grouper(key="searchedDate", freq="H", offset="30min")
).mean()

df_vec["searchedDate"] = pd.to_datetime(df_vec.searchedDate, format = "%Y-%m-%d", utc=True)

df1 = df_vec.groupby(pd.Grouper(key="searchedDate", freq="H", offset="30min")).mean()

df2 = df1[df1["viewCount"].notna()].reset_index()

df_final = pd.read_csv("ssss.csv")

df_final["Time"] = pd.to_datetime(df_final["Time"], format = "%Y-%m-%d", utc=True)

vec4merge = pd.concat([df2.searchedDate, df2.iloc[:, -768:]], axis = 1)

df_final_vec = df_final.merge(vec4merge, left_on = "Time", right_on = "searchedDate")

df_final_vec.drop("Unnamed: 0", axis  = 1, inplace = True)

In [181]:
df_final_vec

,viewCount,likeCount,dislikeCount,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,...,758,759,760,761,762,763,764,765,766,767
0,9.644912e+05,40463.448276,1070.793103,15.241379,84589.429291,3723.437055,97.624292,0.971347,0.971347,82587.974284,...,0.252636,0.014005,0.046532,0.201989,-0.026079,0.054013,0.323389,0.087363,0.020213,0.071755
1,1.045160e+06,44341.740000,1192.020000,17.400000,63909.437389,2967.019734,89.903678,0.969618,0.969618,62169.305249,...,0.266792,-0.013423,0.016061,0.154022,-0.029624,0.091755,0.319605,0.115718,0.027696,0.089908
2,1.077168e+06,46144.080000,1244.080000,18.640000,64870.463707,3002.939559,91.268385,0.967561,0.967561,63097.778622,...,0.244780,-0.033615,-0.033858,0.148356,-0.057002,0.077208,0.319512,0.107262,0.054674,0.078297
3,1.077747e+06,46178.200000,1245.080000,18.680000,65211.201800,3031.484044,92.005868,0.969667,0.969667,63429.337457,...,0.244662,-0.023792,-0.028832,0.136220,-0.052073,0.074929,0.294398,0.090444,0.034019,0.075988
4,1.080107e+06,46259.160000,1248.360000,18.960000,65479.014739,3024.847456,92.011051,0.969174,0.969174,63687.150884,...,0.247546,-0.038550,-0.025746,0.139234,-0.050697,0.086759,0.298814,0.105302,0.040580,0.084385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,8.561130e+05,44029.640000,966.760000,122.960000,13053.192061,783.768560,19.098946,0.958856,0.958856,12706.300806,...,0.001122,0.080423,-0.031749,0.043574,-0.034188,0.075686,0.359191,0.228140,0.084224,-0.064844
794,8.570901e+05,44060.440000,968.200000,123.240000,12421.798377,741.488729,16.643729,0.958569,0.958569,12114.157836,...,0.001122,0.080423,-0.031749,0.043574,-0.034188,0.075686,0.359191,0.228140,0.084224,-0.064844
795,1.007233e+06,50007.560000,1136.760000,131.080000,13108.659614,769.382933,17.351029,0.959595,0.959595,12782.911467,...,0.028876,0.070318,-0.018650,0.051252,-0.037467,0.069777,0.353740,0.215324,0.099823,-0.042077
796,8.577154e+05,44094.920000,969.280000,123.280000,12679.298844,765.462176,17.285382,0.959735,0.959735,12362.124996,...,-0.000363,0.084852,-0.033774,0.078844,-0.042536,0.078368,0.358929,0.215671,0.093998,-0.066405


In [183]:
X2 = df_final_vec.iloc[:,5:].drop(["percent_change_24", "percent_change_48", "percent_change_72", "percent_change_96", 
                             'Time',
       'price', 'percent_change_24', 'percent_change_48', 'percent_change_72',
       'percent_change_96', 'msft', 'amd', 'ndaq', 'intel', 'qualcomm',
       'apple', 'searchedDate'], axis = 1)

In [184]:
X2

,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,videoTitleSentimentScore_vader_pos,videoTitleSentimentScore_vader_com,captionSentimentScore_vader_pos,captionSentimentScore_vader_neg,captionSentimentScore_vader_com,...,758,759,760,761,762,763,764,765,766,767
0,3723.437055,97.624292,0.971347,0.971347,82587.974284,0.277400,-0.014347,0.122280,0.046480,0.998404,...,0.252636,0.014005,0.046532,0.201989,-0.026079,0.054013,0.323389,0.087363,0.020213,0.071755
1,2967.019734,89.903678,0.969618,0.969618,62169.305249,0.319667,-0.081008,0.121952,0.042571,0.998157,...,0.266792,-0.013423,0.016061,0.154022,-0.029624,0.091755,0.319605,0.115718,0.027696,0.089908
2,3002.939559,91.268385,0.967561,0.967561,63097.778622,0.307000,-0.004042,0.120857,0.039333,0.998038,...,0.244780,-0.033615,-0.033858,0.148356,-0.057002,0.077208,0.319512,0.107262,0.054674,0.078297
3,3031.484044,92.005868,0.969667,0.969667,63429.337457,0.319667,-0.082817,0.121143,0.042048,0.998081,...,0.244662,-0.023792,-0.028832,0.136220,-0.052073,0.074929,0.294398,0.090444,0.034019,0.075988
4,3024.847456,92.011051,0.969174,0.969174,63687.150884,0.319667,-0.022800,0.118905,0.041810,0.997852,...,0.247546,-0.038550,-0.025746,0.139234,-0.050697,0.086759,0.298814,0.105302,0.040580,0.084385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,783.768560,19.098946,0.958856,0.958856,12706.300806,0.337000,-0.228971,0.128150,0.041150,0.998830,...,0.001122,0.080423,-0.031749,0.043574,-0.034188,0.075686,0.359191,0.228140,0.084224,-0.064844
794,741.488729,16.643729,0.958569,0.958569,12114.157836,0.337000,-0.228971,0.128150,0.041150,0.998830,...,0.001122,0.080423,-0.031749,0.043574,-0.034188,0.075686,0.359191,0.228140,0.084224,-0.064844
795,769.382933,17.351029,0.959595,0.959595,12782.911467,0.362333,-0.132087,0.129850,0.041050,0.999175,...,0.028876,0.070318,-0.018650,0.051252,-0.037467,0.069777,0.353740,0.215324,0.099823,-0.042077
796,765.462176,17.285382,0.959735,0.959735,12362.124996,0.337000,-0.241762,0.130600,0.040850,0.999190,...,-0.000363,0.084852,-0.033774,0.078844,-0.042536,0.078368,0.358929,0.215671,0.093998,-0.066405


In [158]:
X2.isna().sum().sort_values(ascending = False)[:10]

videoTitleSentimentScore_vader_neg_y                  218
videoTitleSentimentScore_vader_neg_x                  187
caption_list_split_by_nvidia_20words_vaderscores_x      1
caption_list_split_by_nvidia_10words_flairscores_x      1
caption_list_split_by_nvidia_35words_flairscores_y      1
caption_list_split_by_nvidia_10words_vaderscores_y      1
caption_list_split_by_nvidia_20words_vaderscores_y      1
caption_list_split_by_nvidia_35words_vaderscores_y      1
caption_list_split_by_nvidia_35words_vaderscores_x      1
caption_list_split_by_nvidia_10words_flairscores_y      1
dtype: int64

In [131]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [193]:
reg = LinearRegression()
reg.fit(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_24"])
reg.score(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_24"])
pred_24 = reg.predict(df_final_vec.iloc[:,-768:])

In [194]:
reg = LinearRegression()
reg.fit(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_48"])
reg.score(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_48"])
pred_48 = reg.predict(df_final_vec.iloc[:,-768:])

In [195]:
reg = LinearRegression()
reg.fit(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_72"])
reg.score(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_72"])
pred_72 = reg.predict(df_final_vec.iloc[:,-768:])

In [196]:
reg = LinearRegression()
reg.fit(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_96"])
reg.score(df_final_vec.iloc[:,-768:], df_final_vec["percent_change_96"])
pred_96 = reg.predict(df_final_vec.iloc[:,-768:])

In [204]:
vec_pred = pd.DataFrame({'pred_24': pred_24, 
'pred_48': pred_48,
'pred_72': pred_72,
'pred_96': pred_96})

In [206]:
vec_pred.to_csv('vec_pred.csv', index = False)

In [188]:
reg = LinearRegression()
reg.fit(X2, df_final_vec["percent_change_24"])
reg.score(X2, df_final_vec["percent_change_24"])

0.9038540927891954

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, df_final_vec.iloc[:,-768:], df_final_vec[["percent_change_24"]], scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)